### Big Idea:

Statistics modeled in a program are easier to get right and understand than using a formulaic approach. it is also extends to more complicated situations that classic formulas.

Jake VanderPlas: If you can do a for loop, you can do statistics.

In [1]:
from random import *
from statistics import *
from collections import *

### 1. Six roulette wheeles -- 18 red 18 black 2 greens

--> choices with weighting

#### - Tedius way

In [2]:
population = ['red'] * 18 + ['black'] * 18 + ['green'] * 2

In [3]:
Counter([choice(population) for i in range(6)])

Counter({'red': 5, 'green': 1})

In [4]:
# more succinctly
Counter(choices(population, k=6))

Counter({'black': 5, 'red': 1})

#### - Better way

In [6]:
Counter(choices(['red', 'black', 'green'], [18, 18, 2], k=6))

Counter({'red': 3, 'black': 3})

### 2. Deal 20 playing cards without replacement (16 tens, 36 low)

--> Counter, eleoments, sample, list.count

In [11]:
deck = Counter(tens=16, low=36)

In [12]:
deck

Counter({'tens': 16, 'low': 36})

In [13]:
deck = list(deck.elements())

In [14]:
deal = sample(deck,52)
# or shuffle(deck)

In [23]:
remainder = deal[20:]

In [24]:
Counter(remainder)

Counter({'low': 21, 'tens': 11})

### 3. 5 or more heads from 7 spins of a biased coin

--> lambda, choices, list.count

- cumulated weight, internally: the weights have to be accumulated anyway, so it speeds up the simulation a little bit

- lambda with no arguments: a deferred computation -> in the futre run a computation

In [43]:
trial = lambda: choices(['heads', 'tails'], cum_weights=[0.60, 1.00], k=7).count('heads') >=5

In [49]:
n = 100000
sum(trial() for i in range(n))/n       # <== empirical result

0.41968

In [46]:
# compare to the analytic approach
from math import factorial as fact
def comb(n, r):
    return fact(n) // fact(r) // fact(n - r)
# this is floor division: to return the int not the float

In [52]:
ph = 0.6
# 5 heads out of 7 spins
p_5 = ph ** 5 * (1 - ph) ** 2 * comb(7, 5)
# 6 heads out of 7 spins
p_6 = ph ** 6 * (1 - ph) ** 1 * comb(7, 6)
# 7 heads out of 7 spins
p_7 = ph ** 7 * (1 - ph) ** 0 * comb(7, 7)
p_5 + p_6 + p_7   # <== theoretical result

0.419904

### 4. Probability that the median of 5 samples falles a middle quartile

--> chained comparison, choices from a range

In [53]:
trial = lambda: n // 4 < median(sample(range(100000), 5)) <= n * 3 //4

In [54]:
sum(trial() for i in range(n)) / n

0.79337

### 3. Bootstrapping to estimate the confidence interval on a sample of data (timing of a network event)

--> sorted, mean, choices

- Bootstrapping is used when obtaining new samples is expensive

- Stanrd error of the mean relies on a normal distribution. Resampling makes no such assumptions

- Assumption: If the data represnets the population, it can give the as another sample from the population via resampling

In [55]:
timings = [7.18, 8.59, 12.24, 7.39, 8.16, 8.68, 6.98, 8.31, 9.06, 7.06, 7.67, 10.02, 6.87, 9.07]

In [64]:
mean(timings), stdev(timings)

(8.377142857142857, 1.4576505256559458)

In [59]:
def bootstrap(data):
    return choices(data, k=len(data))

In [63]:
# Build a 90% confidence interval
n =  10000
means = sorted(mean(bootstrap(timings)) for i in range(n))
print(f'Falls in a 90% confidence interval from {means[round(n*0.05)] : .3f} to {means[-round(n * 0.05)] : .3f}')

Falls in a 90% confidence interval from  7.797 to  9.032


### Statistical significance of the difference of two means 

--> shuffle, slicing, mean

- p-value: what is the chance that what we observed was actually due to chance, rather than due to a real effect

- we have female and male web visitors and we can observe their behaviors, e.g. different mean amount of time on the website, mean of amount of purchases. Question: Was that a real difference, or just due to chance?

In [66]:
drug = [7.1, 8.5, 6.4, 7.7, 8.2, 7.6, 8.4, 5.1, 8.1, 7.4, 6.9, 8.4]
placebo = [8.2, 6.1, 7.1, 7.1, 4.9, 7.4, 8.1, 7.1, 6.2, 7.0, 6.6, 6.3]

In [67]:
mean(drug), mean(placebo)

(7.483333333333333, 6.841666666666667)

In [70]:
obs_diff = mean(drug) - mean(placebo)
obs_diff

0.6416666666666666

- permutation test--> 
Null hypothesis: the drug did nothing, that there was no difference between placebo and drug --> we should be able to swap participants out of one group and into the another, and the observed mean should be as extreme as the obs_diff
- If we reshuffle (permuting, relabeling) the participants, is the new mean_diff the same or extreme than be observed?
    

In [75]:
comb = drug + placebo
nd = len(drug)

In [80]:
def trial():
    shuffle(comb)
    drug = comb[:nd]
    placebo = comb[nd:]
    new_diff = mean(drug) - mean(placebo)
    return new_diff >= obs_diff

In [84]:
n = 10000
sum(trial() for i in range(n))/n    #^-- p-value: likelihood that the outcome was solely due to chance

0.0566

- p_value = 0.0566: dont accept null-hypothesis, but you believe you need additional data to confirm

### Single server queue simulation

--> expovariate, gauss, mean, median, stdev

- Poisson distribution (expovariate)

In [95]:
average_arrival_interval = 5.6
average_service_time = 5.0
stdev_service_time = 0.5

In [96]:
num_waiting = 0
arrivals = []
starts = []
arrival = service_end = 0.0

In [97]:
for i in range(20000):
    if arrival <= service_end:
        num_waiting += 1
        arrival += expovariate(1.0/ average_arrival_interval)
        arrivals.append(arrival)
    else:
        num_waiting -= 1
        service_start = service_end if num_waiting else arrival
        service_time = gauss(average_service_time, stdev_service_time)
        service_end = service_start + service_time
        starts.append(service_start)

waits = [start - arrival for arrival, start in zip(arrivals, starts)]

In [101]:
print(f'Mean wait: {mean(waits) :.1f}. Stdev wait: {stdev(waits) :.1f}.')
print(f'Median wait: {median(waits) :.1f}. Max wait: {max(waits) :.1f}.')

Mean wait: 24.6. Stdev wait: 27.7.
Median wait: 16.4. Max wait: 187.5.


- Then you can do sensibility test: increase average_service_time, decrease stdev_service_time